# 実験を開始する

実験記録に必要な実験パッケージの用意によって実験の作成と開始をサポートします。  
実験パッケージは、研究リポジトリのexperimentsフォルダ配下に追加されます。  
上から順番に実行してください。  
2回目以降の実行の場合、このセルが選択された状態で画面上部に表示される以下のボタンをクリックしてから実行して下さい。  
![UnfreezeBotton](https://raw.githubusercontent.com/NII-DG/workflow-template/develop/sections/images/unfreeze_button.png)

## 1. 実験環境を作成するための準備をする

In [ ]:
import json
import os

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
# urlを生成する
%cd ~/
remote_http_url = !git config --get remote.origin.url
for item in remote_http_url:
    remote_http_url = item

repository_title = remote_http_url.split('/')[-1].replace('.git', '')

## 2. 実験環境にアクセスする

以下のうち、いずれかの手順を実施してください。

- 標準的な実験活動の場合：「2-A. 標準環境での実験」
- 高性能実験環境を利用する場合：「2-B. 高性能実験環境での実験」

### - 2-A. 標準環境での実験

以下のセルを実行した後に出力されるリンクをクリックして実験に移ってください。

In [ ]:
import urllib

print(
    "https://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(remote_http_url, safe='') + "/HEAD?filepath=WORKFLOWS/experiment.ipynb"
)

### - 2-B. 高性能実験環境での実験

以下のセルを実行し、「高性能実験環境と正常に接続されています」と表示されることを確認してください。  
表示されない場合、[高性能実験環境を準備する](../01_preparation_phase/base_setup_data_analysis_tools.ipynb) の手順を実施してください。

In [ ]:
!ssh mdx "echo 高性能実験環境と正常に接続されています"

高性能実験環境の接続先情報を取得します。以下のセルを実行してください。

In [ ]:
mdx_host = ''
with open('/home/jovyan/.ssh/config', 'r') as f:
    tmp = f.read()
    tmp = tmp[tmp.find('mdx'):]
    tmp = tmp[tmp.find('Hostname ') + len('Hostname '):]
    mdx_host = tmp[:tmp.find('\n')]

高性能実験環境に実験パッケージ情報を転送します。以下のセルを実行してください。

In [ ]:
# コンテナを起動する毎に最新のリポジトリをクローンする
list = !ssh mdx ls
for path in list:
    if path == repository_title:
        !ssh mdx rm -rf $repository_title

cmd = "git clone " + remote_http_url
!ssh mdx $cmd

高性能実験環境においてJupyterLabを起動します。以下のセルを実行してください。  
実行後、表示されるリンクをクリックすると、パスワードの入力が求められますので「gpu-jupyter」と入力してLoginしてください。

In [ ]:
from IPython.display import clear_output

# JupyterLabコンテナイメージの作成
cmd = "docker build -t dg/jupyterlab:1.1 ./" + repository_title + "/"
!ssh mdx $cmd
clear_output()

# JupyterLabコンテナの起動
if mdx_host == '163.220.176.51':
    cmd = "docker run --gpus all -d -P dg/jupyterlab:1.1"
else:
    cmd = "docker run -d -P dg/jupyterlab:1.1"
container_id = !ssh mdx $cmd
container_id = container_id[0][:12]

# ホストマシン上で動的に割り当てられたポートを取得
cmd = "docker ps -l"
port = !ssh mdx $cmd
port = port[1]
port = port[port.find("0.0.0.0:") + 8: port.find("->8888/tcp")]

# JupyterLab利用のためのURLを表示
url = "http://" + mdx_host + ":" + str(port) + "/notebooks/WORKFLOWS/experiment.ipynb"
print(url)

## 3. ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../../base_FLOW.ipynb)からトップページに遷移できます。  